# 0. Env

In [ ]:
from tqdm.auto import tqdm
import requests

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms, models

from datasets import load_dataset, VerificationMode

import matplotlib.pyplot as plt

In [ ]:
# Gradient False
# Pytorch에서 동작을 확안하기 위해서 Gradient 계산을 하지 않도록 설정
torch.set_grad_enabled(False)

# 1. MNIST fc

In [ ]:
# GPU 사용 가능 여부 확인
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

In [ ]:
# 데이터 전처리
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

train_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)

In [ ]:
# 모델 정의
class LinearModel(nn.Module):
    def __init__(self):
        super(LinearModel, self).__init__()
        self.linear = nn.Linear(28*28, 10)  # 28*28 입력, 10 출력 (클래스 수)

    def forward(self, x):
        x = x.view(x.size(0), -1)
        return self.linear(x)

In [ ]:
# 모델 초기화
model = LinearModel()
model.to(device)

# 손실 함수 및 최적화
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [ ]:
# 학습
torch.set_grad_enabled(True)
num_epochs = 10
for epoch in range(num_epochs):
    loss_sum = 0
    for images, labels in tqdm(train_loader):
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        loss_sum += loss.item()

    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss_sum:.4f}')

In [ ]:
# 가중치 시각화
weights = model.linear.weight.data
weights = weights.view(10, 28, 28)
figure, axes = plt.subplots(2, 5, figsize=(12, 6))
for i, ax in enumerate(axes.flat):
    ax.imshow(weights[i].cpu().numpy(), cmap='gray')
    ax.set_title(f'Digit {i}')
    ax.axis('off')
plt.show()

# 2. VGG-16

In [ ]:
# GPU 사용 가능 여부 확인
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

In [ ]:
# 학습 데이터셋
dataset_train = load_dataset(
    'cifar10',
    split='train', # training dataset
    verification_mode=VerificationMode.NO_CHECKS
)

dataset_train

In [ ]:
# 평가 데이터셋
dataset_valid = load_dataset(
    'cifar10',
    split='test',  # test set
    verification_mode=VerificationMode.NO_CHECKS
)

dataset_valid

In [ ]:
# class 개수
num_classes = len(set(dataset_train['label']))
num_classes

In [ ]:
# 데이터 확인
plt.imshow(dataset_train[0]['img'])
plt.show()

In [ ]:
preprocess = transforms.Compose([
    transforms.Resize((32, 32)),  # 이미지를 32x32로 Resize
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [ ]:
inputs_train = []

for record in tqdm(dataset_train):
    image = record['img']
    label = record['label']

    # gray to RGP (입력 형태를 동일하게)
    if image.mode != 'RGB':
        image = image.convert("RGB")
        
    # prepocessing
    input_tensor = preprocess(image)
    
    inputs_train.append([input_tensor, label]) 

In [ ]:
print(len(inputs_train), inputs_train[0][0].shape)

In [ ]:
inputs_valid = []

for record in tqdm(dataset_valid):
    image = record['img']
    label = record['label']

    # gray to RGP (입력 형태를 동일하게)
    if image.mode != 'RGB':
        image = image.convert("RGB")
        
    # prepocessing
    input_tensor = preprocess(image)
    
    inputs_valid.append([input_tensor, label]) 

In [ ]:
print(len(inputs_valid), inputs_valid[0][0].shape)

In [ ]:
# add to dataloaders
dloader_train = torch.utils.data.DataLoader(
  	inputs_train,
    batch_size=64,
    shuffle=True
)

dloader_valid = torch.utils.data.DataLoader(
  	inputs_valid,
    batch_size=64,
    shuffle=False
)

In [ ]:
# vgg16 loading
model = models.vgg16(pretrained=True)  # 학습된 파라미터로 모델 로딩
# model = models.vgg16(pretrained=False)  # 랜덤 파라미터로 모델 로딩
model

In [ ]:
# classifier linear 3 수정
model.classifier[6] = nn.Sequential(
                          nn.Linear(4096, 512),
                          nn.ReLU(), 
                          nn.Dropout(0.5),
                          nn.Linear(512, num_classes))

In [ ]:
# model이 GPU를 사용하도록
model.to(device)

In [ ]:
# loss function
loss_func = nn.CrossEntropyLoss()
# learning rate 
lr = 0.001
# SGD optimizer
optimizer = torch.optim.SGD(model.parameters(), lr=lr) 

In [ ]:
num_epochs = 5
for epoch in range(num_epochs):
    model.train() # train mode

    # 학습
    for i, (images, labels) in enumerate(tqdm(dloader_train)):
        images = images.to(device)
        labels = labels.to(device)

        # forward
        outputs = model(images)
        # loss
        loss = loss_func(outputs, labels)

        # backward & optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    # 평가
    with torch.no_grad():  # no gradient
        model.eval() # eval mode
        correct = 0
        total = 0
        all_val_loss = []

        for images, labels in dloader_valid:
            images = images.to(device)
            labels = labels.to(device)

            # forward
            outputs = model(images)
            # total count
            total += labels.size(0)
            # predicted label
            predicted = torch.argmax(outputs, dim=1)
            # calculate correct
            correct += (predicted == labels).sum().item()
            # calculate the loss
            all_val_loss.append(loss_func(outputs, labels).item())
        # calculate val-loss
        val_loss = sum(all_val_loss) / len(all_val_loss)
        # calculate val-accuracy
        val_acc = 100 * (correct / total)
    print(f"Epoch: {epoch + 1} / {num_epochs}, Val-loss: {val_loss:.4f}, Val-ac: {val_acc:.4f}")